# Creating and Valuing a CDS Index

Example CDS Index Valuation

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.185 - This build:  24 Oct 2020 at 20:12 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



In [14]:
valuationDate = FinDate(7, 2, 2006)
settlementDate = valuationDate.addWeekDays(0)
print(settlementDate)

TUE 07 FEB 2006


## Build Ibor Curve

Build the Ibor curve to value cashflows

In [15]:
depos = []
dcType = FinDayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swapType = FinSwapTypes.PAY
swap1 = FinIborSwap(settlementDate,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = FinIborSwap(settlementDate,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = FinIborSwap(settlementDate,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = FinIborSwap(settlementDate,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = FinIborSwap(settlementDate,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

liborCurve = FinIborSingleCurve(valuationDate, depos, [], swaps)

## Build Index Curve

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

In [16]:
stepInDate = valuationDate

In [17]:
indexCoupon = 0.0048375
maturityDate = FinDate(20, 6, 2010)

cdsMarketContracts = []
cds = FinCDS(valuationDate, maturityDate, indexCoupon)
cdsMarketContracts.append(cds)

recoveryRate = 0.40
indexCurve = FinCDSCurve(valuationDate, cdsMarketContracts,liborCurve,recoveryRate)

## Create CDS Index and Perform Valuation

The CDS Index is treated as a CDS with the coupon equal to the CDS Index coupon

In [18]:
cdsRecovery = 0.40
notional = 10.0 * ONE_MILLION
longProtection = True
indexCoupon = 0.004 

cdsIndexContract = FinCDS(stepInDate, maturityDate, indexCoupon, notional, longProtection)

In [19]:
spd = cdsIndexContract.parSpread(valuationDate, indexCurve, cdsRecovery) * 10000
print("PAR SPREAD", spd)

PAR SPREAD 48.37497289672881


In [20]:
v = cdsIndexContract.value(valuationDate, indexCurve, cdsRecovery)
print("FULL VALUE", v['full_pv'])
print("CLEAN VALUE", v['clean_pv'])

FULL VALUE 27148.41172362311
CLEAN VALUE 32592.856168067578


In [21]:
p = cdsIndexContract.cleanPrice(valuationDate, indexCurve, cdsRecovery)
print("CLEAN PRICE", p)

CLEAN PRICE 99.67407120570428


In [22]:
accruedDays = cdsIndexContract.accruedDays()
print("ACCRUED DAYS", accruedDays)

accruedInterest = cdsIndexContract.accruedInterest()
print("ACCRUED COUPON", accruedInterest)

ACCRUED DAYS 49
ACCRUED COUPON -5444.444444444444


In [23]:
protPV = cdsIndexContract.protectionLegPV(valuationDate, indexCurve, cdsRecovery)
print("PROTECTION LEG PV", protPV)

PROTECTION LEG PV 188260.7326852467


In [24]:
premPV = cdsIndexContract.premiumLegPV(valuationDate, indexCurve, cdsRecovery)
print("PREMIUM LEG PV", premPV)

PREMIUM LEG PV 161112.32096162357


In [25]:
rpv01 = cdsIndexContract.riskyPV01(valuationDate, indexCurve)
print("FULL  RPV01", rpv01['full_rpv01'])
print("CLEAN RPV01", rpv01['clean_rpv01'])

FULL  RPV01 4.027808024040589
CLEAN RPV01 3.8916969129294783


Copyright (c) 2020 Dominic O'Kane